In [ ]:
! pip install autogen-core autogen-ext autogen-ext[ollama] autogen-ext[openai] proton_driver

In [1]:
import os

from proton_driver import client

timeplus_host = os.getenv("TIMEPLUS_HOST") or "localhost"
timeplus_user = os.getenv("TIMEPLUS_USER") or "proton"
timeplus_password = os.getenv("TIMEPLUS_PASSWORD") or "timeplus@t+"

In [2]:
import os
import asyncio

from autogen_core import (
    MessageContext,
    RoutedAgent,
    SingleThreadedAgentRuntime,
    TopicId,
    message_handler,
    type_subscription,
    AgentId,
)
from autogen_core.models import ChatCompletionClient, SystemMessage, UserMessage

from autogen_ext.models.cache import ChatCompletionCache, CHAT_CACHE_VALUE_TYPE
from autogen_ext.cache_store.diskcache import DiskCacheStore
from diskcache import Cache

In [3]:
from dataclasses import dataclass
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelInfo

@dataclass
class Message:
    content: str

support_analysis_agent_topic = "SupportAnalysisAgent"

@type_subscription(topic_type=support_analysis_agent_topic)
class SupportAnalysisAgent(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("A support analysis agent.")
        self._system_message = SystemMessage(
            content=(
                """  you are an agent help to anlysis the support communications,
please analysis the communitcation, and return analysis result in a json object, which contains following information

* category
  based on the first message send from the customer, give the communication a category
  question - when customer is asking for support with a question
  issue - when customer is reporting an issue
  development request - customer is looking for new features that timeplus does not support yet
  information - when customer is just sharing some information
  
* brief
  give a simple sentense to describe what is happening in this conversation
  
* result
  was the problem resolved? return resolved or unresolved

* summary
  provide a summary of the who conversation

* start_time
  when the conversation started, it is at the beging of the first message

* serverity
  how series the issue is, could be high, medium, low

sample output here

```json
{
  "category":"",
  "brief" : "",
  "result" : "",
  "summary" : "",
  ... ...
}
```
"""
            )
        )
        self._model_client = model_client
        self._result = None

    @message_handler
    async def inference(self, message: Message, ctx: MessageContext) -> None:
        prompt = f"{message.content}, please analysis"
        llm_result = await self._model_client.create(
            messages=[
                self._system_message,
                UserMessage(content=prompt, source=self.id.key),
            ],
            cancellation_token=ctx.cancellation_token,
        )
        response = llm_result.content
        self._result = response


class SupportAgent:
    def __init__(self):
        cache_dir = os.path.join(os.getcwd(), ".neutrino_cache")
        os.makedirs(cache_dir, exist_ok=True)  # Ensure the directory exists

        """
        model_info = ModelInfo(
            family="llama3.2:1b",
            function_calling=False,
            json_output=False,
            vision=False,
        )
        
        model_client =  OllamaChatCompletionClient(
            model="llama3.2:1b",
            host="http://host.docker.internal:11434/",
            model_info=model_info
            
        )
        """

        model_client =  OpenAIChatCompletionClient(
            model="gpt-4o-mini",
            temperature=0.0,
        )
        
        cache_store = DiskCacheStore[CHAT_CACHE_VALUE_TYPE](Cache(cache_dir))
        self.client = ChatCompletionCache(model_client, cache_store)
        self.runtime = SingleThreadedAgentRuntime()

    async def _analysis(self, message):

        await SupportAnalysisAgent.register(
            self.runtime,
            type=support_analysis_agent_topic,
            factory=lambda: SupportAnalysisAgent(model_client=self.client),
        )

        self.runtime.start()

        await self.runtime.publish_message(
            Message(content=message),
            topic_id=TopicId(support_analysis_agent_topic, source="default"),
        )

        await self.runtime.stop_when_idle()
        
        inference_agent_id = AgentId(support_analysis_agent_topic, "default")
        inference_agent = await self.runtime.try_get_underlying_agent_instance(
            inference_agent_id
        )

        inference_result = inference_agent._result
        return inference_result

    async def analysis(self, data):
        message = f"based on input data : {data}"
        return  await self._analysis(message)

In [5]:
import re

###### import re
def extract_code_blocks_with_type(markdown_text):
    """
    Extract code blocks and their types from a Markdown string.

    Parameters:
        markdown_text (str): The Markdown content as a string.

    Returns:
        list: A list of tuples, each containing (code_type, code_content).
              If no type is specified, code_type will be an empty string.
    """
    # Regex pattern to match code blocks with or without a code type
    pattern = r"```(\w+)?\n(.*?)```"

    # Use re.DOTALL to capture code content spanning multiple lines
    matches = re.findall(pattern, markdown_text, re.DOTALL)

    # Normalize the results
    return [
        (code_type if code_type else "", code_content.strip())
        for code_type, code_content in matches
    ]


timeplus_client = client.Client(host=timeplus_host, port=8463, user=timeplus_user,password=timeplus_password)

analysis_result_stream = "slack_support_analysis_salla"
## create the analysis stream
try:
    timeplus_client.execute(
        f"""CREATE STREAM IF NOT EXISTS {analysis_result_stream} (
        raw string
    )
    """
    )
except Exception as e:
    print(e)

rows = timeplus_client.execute_iter(f"select * from v_salla_support_merged_threads")
count = 0

results = []
for row in rows:
    input = row[0]
    results.append(input)

print(f'there are {len(results)} conversations')
    

for r in results:
    agent = SupportAgent()
    result = await agent.analysis(r)
    code = extract_code_blocks_with_type(result)
    if len(code) > 0:
        analaysis_result = code[0][1]
        print(analaysis_result)
        try:
            timeplus_client.execute(
                f"INSERT INTO {analysis_result_stream} (raw) VALUES",
                [
                    [
                        analaysis_result
                    ]
                ],
            )
        except Exception as e:
            print(e)
    
    else:
        print(f"failed to parse , input is {input }")



there are 36 conversations
{
  "category": "issue",
  "brief": "The customer reported an urgent issue with the slowness of remote lookup in TimePlus.",
  "result": "unresolved",
  "summary": "Salah Alkhwlani reported an urgent issue regarding the slowness of remote lookup in TimePlus. The team discussed the issue, with Ibrahim Bakhsh performing a patch update to address it. There was a conversation about the remote lookup not pushing down to ClickHouse, and the team planned to have a meeting to further investigate the issue.",
  "start_time": "2025-01-02 22:12:26",
  "severity": "high"
}
{
  "category": "issue",
  "brief": "Customer reports a significant disk space issue related to the write-ahead log and seeks assistance.",
  "result": "resolved",
  "summary": "Ibrahim Bakhsh reported that 1 TB of information was consumed in the write-ahead log, suspecting it to be a bug. The team discussed potential causes, including a query that filled the disk to 100%. They explored commands to flu